# Mini Project 4
Sepehr Abbaspour
610398147

At first, we import 'NLTK' library for tokenisation and achieve stop words list.
The two functions needed are called 'wordpunct_tokenize' and 'stopwords', respectively.
Libraries 'string', sklearn, heap1, rank_bm25, math and matplotlib are imported for better preprocessing, importing vector space structure, max-heap sorting, applying Okapi BM25 algorithm, rounding recalls to tens, and generating random parameters for smoothing in language model, respectively.

In [ ]:
import nltk
from nltk.tokenize import wordpunct_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from heapq import nlargest
from rank_bm25 import BM25Okapi
from math import prod, ceil, floor
from random import uniform

Defining preprocessor function which preprocess the terms in the documents and queries.
In this function the input string is changed in the way of lowering its letters, elimination of the stop words and asymmetric expansion (changing from plural form to singular form in the specific rules).
Tokenisation level is to delete symbolic letters by iterating the full-lowered string.
Some variables are defined to be pointed for the next times.
After that, the terms are devided and assembled to make String be the list of them and each of them gets ready for a possible changing.
The changing contains stop word deletion in which stop words become removed from the list and replaces with null phrase ("").
This replacements exist to avoid changing the length of the list 'String' while iterating it.
After stop words deletion, we check any plural form for each non-stop word term and if its in any plural form, it'll be changed to its singular form according to official plural nouns table rules.
At last the null phrases in the list of edited terms become removed and the final list is returned by the function.
A bit try to preprocess on verbs in different time forms added.

In [ ]:
def preProcessor(String):
    null, Consonants, Irregulars, stopWords = "", "qwrtypsdfghjklzxcvbnm", {"begun": "begin", "frozen": "freeze", "children": "child", "feet": "foot", "teeth": "tooth","mice": "mouse", "people": "person"}, stopwords.words('english')
    String = wordpunct_tokenize(String) #Case Folding & Tokenisations
    String = [Token.lower() for Token in String if Token not in punctuation and Token.lower() not in stopWords]
    lengthString = len(String)
    
    for Index in range(lengthString):
        #Asymmetric Expansion
        if String[Index].endswith("ies") and String[Index][-4] in Consonants:
            String[Index] = String[Index][:-3] + "y"

        elif String[Index].endswith("ves"):
            String[Index] = String[Index][:-3] + "f"
            
        elif String[Index].endswith("es") and (String[Index][-3] in ["s", "x", "z"] or String[Index][-4:-2] in ["ch", "sh"] or (String[Index][-3] == "o" and String[Index][-4] in Consonants)):
            String[Index] = String[Index][:-2]
        
        elif String[Index].endswith("s"):
            String[Index] = String[Index][:-1]
            
        elif String[Index].endswith("men"):
            String[Index] = String[Index][:-3] + "man"

        if String[Index].endswith("ed"):
            String[Index] = String[Index][:-2]
            
        elif String[Index] in Irregulars.keys():
            String[Index] = Irregulars[String[Index]]

        elif String[Index].endswith("en"):
            String[Index] = String[Index][:-2]
 
    while null in String:
        String.remove(null)
        
    return String


Now we're gonna contruct model functions

The first function uses space vector model to rank the documents according to the input query and outputs the k first of best relevant documents by means of their given ranks.

In [ ]:
def spaceVector(Query, Bound):
    tfidfVectorizer, q, vectorScore, retrievedDocs = TfidfVectorizer(), [" ".join(Query)], {}, []
    d = tfidfVectorizer.fit_transform(Documents)
    q = tfidfVectorizer.transform(q)
    Scores = cosine_similarity(q, d)
    for Index in range(1400):
        vectorScore[Index] = Scores[0][Index]
        
    Ids, Scores = list(vectorScore.keys()), list(vectorScore.values())
    electedScores = nlargest(Bound, Scores) #Max-heap implemention!
    return [{"Id": Ids[Scores.index(Score)] + 1, "Score": Score} for Score in electedScores] #Retrieved documents.

The second function uses probabilistic model to rank the documents according to the input query and outputs the k first of best relevant documents by means of their given ranks.

In [ ]:
def Probabilistic(Query, Bound):
    BMProb = BM25Okapi(InvertedIndex)
    Scores, electedDocuments = BMProb.get_scores(Query), BMProb.get_top_n(Query, Documents , Bound) #Also handles large queries by built-in funtions and methods.
    return [{"Id": Documents.index(Document) + 1, "Score": Scores[Documents.index(Document)]} for Document in electedDocuments] #Retrieved documents.

The third function uses probabilistic model to rank the documents according to the input query and outputs the k first of best relevant documents by means of their given ranks.
It generate some random parameters for Jelinek-Mercer smoothing called 'Lambda'.
It has an unkown error while it can't rank more than 470 first documents.
We're suggested to use higher Lambda for short queries (considered less than 100 contained words.)

In [ ]:
def Unigram(Query, Bound):
    if len(Query) < 100:
        Lambda = uniform(0.1, 1)
    else:
        Lambda = uniform(0, 0.1)
                
    Scores = [prod([Lambda * InvertedIndex[Index].count(Term) / len(InvertedIndex[Index]) + (1 - Lambda) * sum([Document.count(Term) for Document in InvertedIndex]) / TMc for Term in Query]) for Index in range(470)] #Unknown zero division error! 470 is the maximum number of documents in order to be considered in unigram language model.
    electedScores = reversed(sorted(Scores)[-Bound:])
    return [{"Id": Scores.index(Score) + 1, "Score": Score} for Score in electedScores] #Retrieved documents.

This function evaluate 11-point interpolated average precision with a given list of ranked documents designated by one three models.
The recalls ar eall rounded to the nearest tenth in range 0 and 1.

In [ ]:
def Evaluation(Index, retrievedDocs):
    Coordinates, AP = {}, 0
    RDocs = set(Relevants[Index]) #Relevant items for the query
    numberRDocs = len(RDocs)
    for Measure in range(Bound): #Iterate on retrieved documents index by index ascendingly and then, calculate precision and recall and make coordinate '(Precision, Recall)' by the whole considered documents in each iterate.
        Measures = retrievedDocs[:Measure + 1] 
        tp = len(set(Measures).intersection(RDocs))
        Recall = tp / numberRDocs
        if Recall % 1000 // 100 >= 5:
            Recall = ceil(Recall * 10) / 10
        else:
            Recall = floor(Recall * 10) / 10
            
        if Recall not in Coordinates:
            Coordinates[Recall] = [tp / len(Measures)]
            continue
        
        Coordinates[Recall].append(tp / len(Measures))

    Recalls = Coordinates.keys()
    for Recall in Recalls:
        AP += max(Coordinates[Recall])
        
    return AP / 11

Here are defined global variables used in the functions; and also a huge list contains documents and another huge list contains whole queries.
There're also list of really relevant documents twoard (index + 1)th query from the list of queries.
These data structures must relate to opened and read test data.

In [ ]:
#Globals!
docFile, qFile, qrFile, Relevants, nonRelevants, SVAPs, Okapi25APs, UniAPs, TMc = open("cran.all.1400").read(), open("cran.qry").read(), open("cranqrel.txt"), {}, {}, [], [], [], 0
InvertedIndex, Queries = docFile.split(".I ")[1:], qFile.split(".I ")[1:] #First element will be '[]'!
#Preprocessing ONLY TITLE and TEXT!
InvertedIndex, Queries = [preProcessor(Document[Document.find(".T") + 2 : Document.find(".A")] + " " + Document[Document.find(".W") + 2:]) for Document in InvertedIndex], [preProcessor(Query[7:]) for Query in Queries]
#Total number of tokens in the collection.
for Document in InvertedIndex:
    TMc += len(Document)
    
Documents = [" ".join(Document) for Document in InvertedIndex]
for Line in qrFile:
    Line = list(map(int, Line.split()))
    if Line[2] >= 1:
        queryTopic = Line[0]
        if queryTopic in Relevants:
            Relevants[queryTopic].append(Line[1])
            continue
        
        Relevants[queryTopic] = [Line[1]]
        continue
    
    nonRelevants[queryTopic] = Line[1]

qrFile.close()
while True:
    try:
        Bound = int(input("How many documents do you need them be retrieved? "))
        if Bound >= 0:
            break

    except ValueError:
        print("Enter a non-negative integer!")

print(f"{InvertedIndex}\n")
print(f"{Documents}\n")
print(f"{Relevants}\n")
print(f"{nonRelevants}\n")

Iterating the list of queries and evaluate whatever needed and then evaluate MAP for each model.
The bigger MAP does a model have, the better model for a great Information Retrieval system it is.

In [ ]:
for Query in Queries:
    SV, Okapi25, Uni, realqIndex = spaceVector(Query, Bound), Probabilistic(Query, Bound), Unigram(Query, Bound), Queries.index(Query) + 1
    print(f"Space Vector Model retrieved {Bound} documents may be relevant with {realqIndex}th query: {SV}\n")
    print(f"Probabilistic Model 'Okapi BM25' retrieved {Bound} documents may be relevant with {realqIndex}th query: {Okapi25}\n")
    print(f"Smoothed Unigram Language Model {Bound} documents may be relevant with {realqIndex}th query: {Uni}\n")
    
    SVAPs.append(Evaluation(realqIndex, [Retrieved["Id"] for Retrieved in SV]))
    Okapi25APs.append(Evaluation(realqIndex, [Retrieved["Id"] for Retrieved in Okapi25]))
    UniAPs.append(Evaluation(realqIndex, [Retrieved["Id"] for Retrieved in Uni]))

print(f"11-point interpolated average precision for the whole queries {SVAPs}\n")
print(f"MAP = {sum(SVAPs) / len(SVAPs)}\n")
print(f"11-point interpolated average precision for the whole queries {Okapi25APs}\n")
print(f"MAP = {sum(Okapi25APs) / len(Okapi25APs)}\n")
print(f"11-point interpolated average precision for the whole queries {UniAPs}\n")
print(f"MAP = {sum(UniAPs) / len(UniAPs)}\n")

This program is in an optimal form due to the pointed conditions in the constructions of the defined functions. 